#Data preprocessing

https://www.kaggle.com/datasets/ankurzing/aspect-based-sentiment-analysis-for-financial-news

The news headlines data set contains information outside of the scope of our research: news headlines affecting more than one entity (some of the rows), name of entity that headline affect (comapny name in col C), and number of words in a headline (col D), and number of news (col A) . With the following code, we remove this information and write processed data into a new file.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import csv
import json

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Functions that remove redundant information from the data

In [2]:
#Headlines affecting >1 company in column C have >1 colon
def has_two_colons(data):
    return data.count(':') >= 2

#Standardize labels tags
def label_to_numeric(label):
    if label == 'neutral':
        return 1
    elif label == 'positive':
        return 2
    elif label == 'negative':
        return 0
    else:
      return None #in case unknow labels

#Extract and return values in dictionaries in col C
def extract_values(json_data):
    data = json.loads(json_data)
    # Extract and convert labels to numeric values
    values = [label_to_numeric(value) for value in data.values()]
    return values

#Combine plus remove rows with headlines affecting >1 company
def remove_rows_with_two_colons(input_file, output_file):
    #Open input and output fils
    with open(input_file, 'r', newline='') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames[1:-1]  # Exclude the first and last column
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        # Write header
        writer.writeheader()

        # Process each row
        for row in reader:
            if not has_two_colons(row['Decisions']):
              row['Decisions'] = extract_values(row['Decisions'])[0] #Leave only (standardized) labels from 'Decisions' column
              del row[reader.fieldnames[0]]  # Delete the first column
              del row[reader.fieldnames[-1]]  # Delete the last column
              writer.writerow(row)

Rewrite data into new file

In [3]:

input_file = '/content/drive/MyDrive/Turing Machine Learining/Sentiment_Analysis/News_headlines_tagged_uncleaned.csv'
output_file = '/content/drive/MyDrive/Turing Machine Learining/Sentiment_Analysis/News_headlines_tagged.csv'

remove_rows_with_two_colons(input_file, output_file)
